In [ ]:
import pandas as pd
from scipy.spatial import cKDTree
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 격자 및 인프라 데이터 불러오기
grid_path = '/content/drive/MyDrive/data/park.csv'
infra_path = '/content/drive/MyDrive/data/ani.csv'

grid_df = pd.read_csv(grid_path)
infra_df = pd.read_csv(infra_path)

# 격자 중심점 좌표 생성
grid_df['geometry'] = grid_df.apply(lambda row: Point(row['경도'], row['위도']), axis=1)

# KDTree를 이용하여 가장 가까운 인프라 지수 및 거리 찾기
infra_tree = cKDTree(infra_df[['Longitude', 'Latitude']].values)
nearest_infra_distances, nearest_infra_indices = infra_tree.query(grid_df[['경도', '위도']].values)

# 거리를 기반으로 인프라 지수 계산 (예시: 거리가 가까울수록 높은 점수)
max_distance = nearest_infra_distances.max()
grid_df['Nearest_Infra_Index'] = max_distance - nearest_infra_distances

# Min-Max 스케일링
scaler = MinMaxScaler()
grid_df['Scaled_Infra_Index'] = scaler.fit_transform(grid_df[['Nearest_Infra_Index']])

# 결과 파일 저장
output_csv_path = '/content/drive/MyDrive/data/park_with_infra.csv'
grid_df.to_csv(output_csv_path, index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
